# **Deep Research (DDG-Powered Agentic Pipeline)**

This project is a **small but complete agentic research pipeline** that plans web searches, gathers **high-signal snippets** via DuckDuckGo, and synthesizes a **long-form report** with inline citations.  

It also includes:
- A **Gradio UI** for interactive use.
- An optional **SendGrid email step** to deliver the final report as HTML.

---

## 🚀 **How It Works (Bird’s-Eye View)**

1. **`PlannerAgent`** — Analyzes your topic and proposes exactly **HOW_MANY_SEARCHES** focused queries, with clear reasoning for each.
2. **Search Agent (DuckDuckGo)** — Runs each query via a tool, producing compact summaries with a “Sources” list.
3. **`WriterAgent`** — Combines mini-summaries into a long report with inline citations `[1]`, `[2]`, etc., and a matching `final_citations` list.
4. **(Optional) Email Agent** — Sends the report in **nicely formatted HTML**.
5. **Gradio UI** — Streams progress and allows downloading the final Markdown.

---

## 🧠 **Agents at a Glance**

### **1. PlannerAgent — “What should I search?”**
- Generates **3 targeted search phrases** to cover:
  - Background
  - Trends
  - Data
  - Breadth
- Ensures maximum relevance and minimal overlap.

---

### **2. Search Agent (DuckDuckGo) — “Get me credible snippets quickly.”**
- Executes `ddg_search` **exactly once** per phrase.
- Summarizes in a few paragraphs.
- Appends a **small, credible Sources list**.

---

### **3. WriterAgent (Synthesizer) — “Turn everything into a coherent report.”**
- Builds a **multi-section Markdown report** (target length: **3–6k words**).
- Adds **inline citations** `[n]` that correspond to an **ordered final_citations** list.

---

### **4. Email Agent — “Send it.”**
- Uses **SendGrid** to deliver the final HTML-formatted report directly to recipients.

---

## 🎨 **User Experience**
- **Interactive Gradio UI** to visualize progress.
- Live streaming of agent outputs.
- **Downloadable Markdown** for offline use.
- Optional **email delivery** for easy sharing.


In [14]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool, ModelSettings
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import List, Dict, Any
from IPython.display import display, Markdown
import os, asyncio, textwrap
from typing import List, Dict, Optional, Any
from pydantic import BaseModel, Field
import gradio as gr
from ddgs import DDGS



In [15]:
# # Fix TLS issues in some environments (VS Code / Windows / corp proxies)

import os, certifi
os.environ["SSL_CERT_FILE"] = certifi.where()  

import sendgrid

In [16]:
load_dotenv(override=True)

True

In [17]:
# ======================================================================================
# 1) DuckDuckGo tool
# ======================================================================================
ddg = DDGS()

# ---------- DDG tool ----------
@function_tool
def ddg_search(query: str,
               max_results: int = 10,
               region: str = "us-en",
               safesearch: str = "moderate") -> List[Dict[str, Any]]:
    """
    Run a DuckDuckGo search and return a list of results.
    Each result includes: title, url, snippet.
    """
    hits = ddg.text(
        query=query,
        max_results=max_results,
        region=region,
        safesearch=safesearch,
        verbose=True
    )
    results: List[Dict[str, Any]] = []
    for h in hits:
        url = h.get("href") or h.get("url") or ""
        if not url.startswith("http"):
            continue
        results.append({
            "title": h.get("title", "") or "",
            "url": url,
            "snippet": h.get("body") or h.get("snippet") or ""
        })
    return results

# ---------- Search agent (uses DDG tool) ----------
SEARCH_INSTRUCTIONS = (
    "You are a research assistant. Given a search term, call `ddg_search` exactly once to fetch results, "
    "then produce a elaborate summary in 4-5 paragraphs (>=1000 words). Capture only the key points—no fluff. "
    "Use the result titles and snippets only; do not browse further.\n\n"
    "After the summary, add a 'Sources' section listing 2–4 of the most relevant items as:\n"
    "[1] Title — URL\n[2] Title — URL\n"
)

search_agent = Agent(
    name="Search agent (DuckDuckGo)",
    instructions=SEARCH_INSTRUCTIONS,
    tools=[ddg_search],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)


In [18]:
# message = "Latest AI Agent frameworks in 2025"

# with trace("Search"):
#     result = await Runner.run(search_agent, message)

# display(Markdown(result.final_output))

In [19]:
# ======================================================================================
# 2) PlannerAgent — structured outputs (queries with reasons)
# ======================================================================================
HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."


class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [20]:

# message = "Latest AI Agent frameworks in 2025"

# with trace("Search"):
#     result = await Runner.run(planner_agent, message)
#     print(result.final_output)

In [21]:
# ======================================================================================
# 3) Email agent (optional)
# ======================================================================================


INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""


@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("garodisk@mail.uc.edu") # Change this to your verified email
    to_email = To("saketgarodia1@gmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}



email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"), 
)


In [22]:
# ======================================================================================
# 4) WriterAgent (Synthesizer) — long-form report with citations
# ======================================================================================
# 
WRITER_INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive, deep report.\n"
    "You will be given the original query and a set of mini-summaries produced by assistants. "
    "Each mini-summary ends with a 'Sources' list. Use ONLY the information in those summaries and sources; "
    "do not invent facts.\n\n"
    "Tasks:\n"
    "1) Write an outline (headings only) for the report.\n"
    "2) Write a long-form Markdown report (aim 3,000–6,000 words) with clear headings/subheadings. "
    "Synthesize across the summaries, compare viewpoints, highlight agreements/disagreements, and avoid duplication. "
    "Where you reference specific facts, insert inline citations like [1], [2], etc. Map these to the ordered list "
    "of final citations you provide at the end.\n"
    "3) Provide a 3–5 sentence executive summary.\n"
    "4) Provide 3–6 follow-up research topics.\n\n"
    "Citation rules:\n"
    "- Only cite items that appear in the provided 'Sources' lists. Use the exact URLs.\n"
    "- Ensure the first time you cite a source in the body, it becomes [1], then [2], etc., in order of first mention. "
    "Your `final_citations` must match that order exactly. Please don't forget to cite sources in the end of the report\n"
)

class Citation(BaseModel):
    url: str = Field(description="Exact source URL used in the report")
    title: Optional[str] = Field(default=None, description="Page/article title, if known")

class ReportData(BaseModel):
    short_summary: str = Field(description="Executive summary (3–5 sentences).")
    outline_markdown: str = Field(description="Markdown outline (headings and bullets).")
    markdown_report: str = Field(description="Full long-form Markdown report with inline [n] citations.")
    follow_up_questions: List[str] = Field(description="3–6 suggested topics to research further.")
    final_citations: List[Citation] = Field(
        description="Ordered list of sources used (maps to [1..N])."
    )

writer_agent = Agent(
    name="WriterAgent (Synthesizer)",
    instructions=WRITER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [23]:
# ======================================================================================
# 5) Small console helpers (pretty progress)
# ======================================================================================

RESET = "\x1b[0m"
BOLD  = "\x1b[1m"
DIM   = "\x1b[2m"
CYAN  = "\x1b[36m"
GREEN = "\x1b[32m"
YELL  = "\x1b[33m"
MAG   = "\x1b[35m"

def info(msg):    print(f"{CYAN}ℹ️  {msg}{RESET}")
def step(msg):    print(f"{GREEN}✅ {msg}{RESET}")
def warn(msg):    print(f"{YELL}⚠️  {msg}{RESET}")
def doing(msg):   print(f"{MAG}🔎 {msg}{RESET}")


In [24]:
# ======================================================================================
# 6) Orchestration helpers
# ======================================================================================

async def plan_searches(query: str) -> WebSearchPlan:
    result = await Runner.run(planner_agent, f"Query: {query}")
    plan = result.final_output
    # pretty print
    info("Planned search phrases:")
    for i, it in enumerate(plan.searches, 1):
        print(f"  {i}. {BOLD}{it.query}{RESET} {DIM}— {it.reason}{RESET}")
    return plan

async def search_one(item: WebSearchItem) -> str:
    # You can include the reason in the prompt for context, but the agent will call the tool.
    doing(f"Searching: {item.query}")
    input_msg = f"Search term: {item.query}\nReason: {item.reason}"
    result = await Runner.run(search_agent, input_msg, max_turns=4)
    step(f"Done: {item.query}")
    return result.final_output  # a Markdown 2–3 paragraph summary + Sources list

async def perform_searches(plan: WebSearchPlan) -> List[str]:
    # Run them concurrently
    tasks = [asyncio.create_task(search_one(it)) for it in plan.searches]
    return await asyncio.gather(*tasks)

def build_writer_input(original_query: str, mini_summaries: List[str]) -> str:
    # Plain text payload, no JSON. Clear delimiters.
    blocks = []
    blocks.append(f"Original query:\n{original_query}\n")
    blocks.append("=== MINI-SUMMARIES START ===")
    for i, s in enumerate(mini_summaries, 1):
        blocks.append(f"\n--- Summary {i} ---\n{s}\n")
    blocks.append("=== MINI-SUMMARIES END ===\n")
    return "\n".join(blocks)

async def write_report(original_query: str, mini_summaries: List[str]) -> ReportData:
    payload = build_writer_input(original_query, mini_summaries)
    result = await Runner.run(writer_agent, payload, max_turns=4)
    return result.final_output

In [25]:
# ======================================================================================
# 7) Actual Research
# ======================================================================================

async def run_research(query: str):
    with trace("Deep Research (DDG)"):
        print("Planning...")
        plan = await plan_searches(query)

        print("Searching...")
        mini_summaries = await perform_searches(plan)

        print("Synthesizing...")
        report = await write_report(query, mini_summaries)

        # Optional: await Runner.run(email_agent, report.markdown_report)
        return plan, mini_summaries, report

In [26]:
# ======================================================================================
# 8) Notebook helpers (pretty rendering of results)
# ======================================================================================


from IPython.display import Markdown, display

def show_report(report: ReportData, mini_summaries: List[str] | None = None):
    # Body
    display(Markdown(report.markdown_report))

    # References (if present)
    if getattr(report, "final_citations", None):
        lines = []
        for i, c in enumerate(report.final_citations, 1):
            title = (c.title or c.url).replace("[", r"\[").replace("]", r"\]")
            lines.append(f"{i}. [{title}]({c.url})")
        display(Markdown("### References\n\n" + "\n".join(lines)))
    else:
        display(Markdown("_No references returned by the writer._"))

def show_plan(plan: WebSearchPlan):
    rows = [f"{i}. **{it.query}** — {it.reason}" for i, it in enumerate(plan.searches, 1)]
    display(Markdown("### Planned search phrases\n\n" + "\n".join(rows)))

def show_mini_summaries(mini_summaries: List[str]):
    if not mini_summaries:
        display(Markdown("_No mini-summaries produced._"))
        return
    parts = ["### Mini-summaries"]
    for i, s in enumerate(mini_summaries, 1):
        parts.append(f"<details><summary><b>Summary {i}</b></summary>\n\n{s}\n\n</details>")
    display(Markdown("\n\n".join(parts)))


In [27]:
# ======================================================================================
# 9) Example usage (Jupyter: just `await main()`; script: `asyncio.run(main())`)
# ======================================================================================

plan, mini_summaries, report = await run_research("List of AI Agent frameworks  sorted by the top ones")
show_plan(plan)
show_mini_summaries(mini_summaries)
show_report(report, mini_summaries)


Planning...
ℹ️  Planned search phrases:
  1. top AI agent frameworks 2023 — To find authoritative and comprehensive lists of AI agent frameworks ranked by usage, popularity, or functional capability.
  2. best AI agent frameworks comparison — To discover community discussions and expert opinions on the most effective AI agent frameworks currently available.
  3. AI frameworks ranking performance 2023 — To gather metrics and evaluation criteria that rank AI agent frameworks based on performance and features.
Searching...
🔎 Searching: top AI agent frameworks 2023
🔎 Searching: best AI agent frameworks comparison
🔎 Searching: AI frameworks ranking performance 2023
✅ Done: best AI agent frameworks comparison
✅ Done: top AI agent frameworks 2023
✅ Done: AI frameworks ranking performance 2023
Synthesizing...


### Planned search phrases

1. **top AI agent frameworks 2023** — To find authoritative and comprehensive lists of AI agent frameworks ranked by usage, popularity, or functional capability.
2. **best AI agent frameworks comparison** — To discover community discussions and expert opinions on the most effective AI agent frameworks currently available.
3. **AI frameworks ranking performance 2023** — To gather metrics and evaluation criteria that rank AI agent frameworks based on performance and features.

### Mini-summaries

<details><summary><b>Summary 1</b></summary>

The search for the top AI agent frameworks in 2023 reveals a variety of options tailored for different use cases, emphasizing functionality, ease of use, and advanced capabilities. Notable frameworks such as AutoGen, LangChain, and CrewAI have emerged as key players, each serving specific needs in the realm of AI development and deployment.

One of the most compelling entrants in the AI agent landscape is Microsoft’s **AutoGen**, which is designed for multi-agent collaborative tasks. Its event-driven architecture allows it to handle complex interactions effectively. Since its launch in September 2023, it has garnered significant attention, achieving over 45,000 stars on GitHub. Organizations like Novo Nordisk have adopted AutoGen, notably due to its distinguished performance on benchmarks like GAIA, which assesses the effectiveness of collaborative AI solutions over traditional single-agent systems. This framework emphasizes not merely performance but the ability to manage multiple agents working together harmoniously, making it especially suitable for enterprise-level applications.

The **LangChain** framework stands out as one of the most powerful tools for building AI agents, particularly those powered by large language models (LLMs). It allows developers to create sophisticated workflows by chaining together prompts, models, memory, and external tools, thus facilitating dynamic and versatile agent capabilities. Developers find LangChain's flexibility advantageous, as it integrates various components into streamlined workflows that enhance the efficiency of building intelligent systems. With its increasing adoption, LangChain is positioned as a frontrunner in the AI agent framework space, heralding profound implications for how developers approach agent-based solutions.

On the more user-friendly side, **CrewAI** is designed with beginners in mind. Its no-code interface makes it accessible for rapid prototyping, allowing users to deploy AI agents quickly without requiring extensive programming knowledge. Equipped with ready-made templates, CrewAI reduces the entry barriers for organizations looking to leverage AI capabilities without a steep learning curve. This accessibility makes it particularly appealing for small businesses or startups, which may lack the resources to invest heavily in sophisticated AI development.

Additionally, the **IBM AI Agent Frameworks** provide a range of choices suitable for different businesses. They offer both beginner-friendly options like CrewAI and more complex frameworks for seasoned developers, like LangGraph. This tiered structure allows businesses to select the framework that best matches their operational needs and technical expertise. Guides and insights from IBM help organizations navigate this choice, ensuring they understand the capabilities and limitations of each framework to make informed decisions.

A comprehensive comparison of these frameworks reveals that the best choice often hinges on the specific use cases, team capabilities, and desired functionalities. In practical terms, organizations must consider whether they need a simple, lower-barrier solution or a multi-faceted, powerful framework capable of handling complex tasks and collaboration among multiple agents.

In summary, the landscape of AI agent frameworks in 2023 is diverse, catering to a wide range of user needs from beginner-friendly systems to advanced collaborative tools capable of tackling complex tasks. The increasing complexity and specificity of these frameworks reflect the rapidly advancing nature of AI technology and the pressing need for effective solutions across industries.

### Sources
[1] Top 9 AI Agent Frameworks as of July 2025 | Shakudo — https://www.shakudo.io/blog/top-9-ai-agent-frameworks  
[2] Top 7 Free AI Agent Frameworks | Botpress — https://botpress.com/blog/ai-agent-frameworks  
[3] The Best AI Agents in 2025: Tools, Frameworks, and Platforms Compared | DataCamp — https://www.datacamp.com/blog/best-ai-agents  
[4] AI Agent Frameworks: Choosing the Right Foundation for Your Business | IBM — https://www.ibm.com/think/insights/top-ai-agent-frameworks  

</details>

<details><summary><b>Summary 2</b></summary>

Exploring the landscape of AI agent frameworks reveals a wealth of options that cater to diverse needs across various applications. Current discussions highlight several promising contenders, among which LangChain, AutoGen, and CrewAI are frequently noted for their robust capabilities. Each framework offers unique features, targeting specific use cases and catering to different user expertise levels. Evaluating these frameworks involves looking at their strengths in multi-agent collaboration, flexibility, usability, and integration into existing systems.

### Key Frameworks and Their Features

1. **LangChain**: This open-source framework stands out due to its strong emphasis on creativity and versatility for AI developers. It provides an extensive library that supports complex applications and interacts seamlessly with various tools, enabling rich workflows and agent interactions. LangChain is particularly well-suited for AI applications in enterprise environments, thanks to its modular design that allows developers to customize functionalities according to specific requirements.

2. **AutoGen**: Developed by Microsoft, AutoGen is praised for its straightforwardness and accessibility, making it an excellent choice for users with varying levels of technical expertise. It supports automation tasks effectively, allowing for quick deployment of AI agents. Detailed comparisons underscore its balance between ease of use and the rich feature set it provides, making it a viable choice for both developers and businesses seeking to leverage AI in operational tasks.

3. **CrewAI**: Known for its strong collaborative capabilities, CrewAI is tailored for projects that require multiple AI agents working in tandem. This framework is lauded for handling complex interactions among agents, making it ideal for applications where teamwork and coordination are paramount. Users appreciate its user-friendly interface and the depth of documentation available, which helps new users get up and running quickly.

### Comparative Insights and Recommendations

When evaluating these frameworks, experts stress the importance of aligning the choice with specific use cases and technical capabilities. For instance, while LangChain excels in applications requiring intricate workflows, users dealing with simpler tasks may favor AutoGen for its ease of use. Similarly, teams looking to create collaborative environments may find CrewAI's features indispensable. The choice often hinges not only on the technical specifications of the frameworks but also on the long-term goals for AI integration within enterprises.

In addition to the leading frameworks, recent analyses also include comparisons of emerging platforms, such as MetaGPT and various alternatives discussed in forums. These alternatives are useful in niches where mainstream frameworks may not offer the required depth or flexibility. For developers focused on cutting-edge applications, keeping tabs on these rising stars ensures they remain at the forefront of technology advancements.

The consensus among experts emphasizes a modular architecture as a critical feature for any AI agent framework, allowing developers to build and adapt their applications dynamically. Furthermore, compatibility with large language models (LLMs) and real-time performance capabilities are highlighted as essential for interactive AI applications that rely heavily on user engagement and instantaneous decision-making.

### Conclusion

The discussions around the best AI agent frameworks showcase a vibrant ecosystem characterized by innovation and adaptability. By considering the distinct strengths and applications of top frameworks such as LangChain, AutoGen, and CrewAI, users can make informed choices that align with their projects' objectives. As technology continues to evolve, staying updated on comparisons will be crucial for anyone involved in AI development or implementation to ensure they leverage the best tools available for their specific contexts.

### Sources
[1] AI agent frameworks : which one is best ? — https://www.linkedin.com/pulse/ai-agent-frameworks-which-one-best-finecortex-lajve  
[2] Best Agentic AI Frameworks Compared 2025 Guide - tkxel — https://tkxel.com/blog/best-agentic-ai-frameworks-comparison/  
[3] 11 Best AI Agent Alternatives & Competitors (2025) — https://manus.so/post/jje032hz  
[4] 10 Top AI Agent Frameworks Unveiled for 2025 - Fusion Chat — https://fusionchat.ai/news/10-top-ai-agent-frameworks-unveiled-for-2025  

</details>

<details><summary><b>Summary 3</b></summary>

In 2023, numerous AI frameworks emerged as dominant tools for developers and researchers, catering to a wide range of machine learning and deep learning tasks. The key frameworks that continue to shape the field include TensorFlow, PyTorch, Keras, and Scikit-Learn. Each framework comes with its own strengths, features, and applications, making it essential to understand their performance metrics thoroughly when evaluating which to use for specific AI projects.

### Key Frameworks and Their Features

**TensorFlow** remains one of the most popular choices among AI developers. It is an open-source library primarily designed for deep learning, but it is versatile enough to support a broader range of machine learning tasks. TensorFlow's comprehensive ecosystem includes libraries for various parts of the AI workflow, such as Keras for simplifying neural network constructs and TensorFlow Lite for mobile and embedded applications. In terms of performance, it is known for its scalability and extensive community support, which is vital for troubleshooting and improvements.

**PyTorch** has also gained substantial traction, especially in the research community. Its dynamic computation graph offers greater flexibility, making it easier for experimenting with novel architectures. Many practitioners favor PyTorch for its intuitive design and ease of use, which can significantly enhance productivity when developing and refining deep learning models. In benchmark tests, PyTorch often competes closely with TensorFlow, demonstrating similar or sometimes superior performance, particularly for specific applications like natural language processing.

**Keras**, now integrated as an official API into TensorFlow, is an abstraction layer that simplifies building and training deep learning models. Its user-friendly interface allows developers, particularly those new to AI, to prototype models quickly. Keras is focused on enabling fast experimentation and is widely appreciated for its straightforward documentation and ease of understanding. This makes Keras an ideal choice for those who prioritize development speed and simplicity without compromising on model performance.

Another important framework is **Scikit-Learn**, which is specifically designed for classical machine learning algorithms. Scikit-Learn excels in pre-processing data, model selection, and evaluation, making it a go-to choice for data scientists working with standard analytical tasks rather than deep learning. Its performance is consistently validated across various benchmarks, providing accessible implementations of algorithms. This framework's efficiency and user-friendly features are vital for practitioners aiming to produce reliable models with less complexity.

### Performance Metrics and Evaluation Criteria

When evaluating AI frameworks, specific performance metrics and evaluation criteria are critical to define which framework suits a project best. Speed, scalability, and ease of deployment are primary metrics. **Speed** refers to the framework’s ability to process large datasets quickly, which is crucial for deep learning models that require extensive computations. Benchmarks often gauge how fast frameworks can train models and handle updates, contributing to their overall reliability.

**Scalability** is another essential factor, particularly for applications requiring robust solutions that can adapt to growing data sizes and algorithmic complexity. TensorFlow, for instance, is designed to scale easily from a single device to distributed setups across multiple machines, which is beneficial for large organizations or massive datasets. **Ease of deployment** is also regarded highly; frameworks facilitating straightforward export to various environments (e.g., mobile, cloud) gain an edge in practical implementations.

**Community support** and **documentation** also play a vital role in the evaluation process. An active community can provide resources, plugins, and troubleshooting experiences, augmenting the framework's capabilities. Detailed and accessible documentation encourages user empowerment, which is especially important for newcomers to AI development.

### Emerging Trends and Comparative Insights

As frameworks evolve, new trends are emerging. A relatively new concept is the integration of different frameworks and tools to unify the strengths of multiple libraries. This trend seeks to address specific shortfalls of one framework with the assets of another, encouraging a more modular approach to model development. Additionally, the rise of **AI agents**, which automate AI processes, represents another paradigm shift currently capturing attention. Platforms like LangChain and CrewAI are being discussed as effective environments for developing complex agents that harness the power of established frameworks, promoting greater efficiency in operations.

Evaluating the effectiveness of these frameworks in 2023 leads to frequently updated benchmarks that inform developers on which tools remain competitive or are fast becoming industry standards. Reports and surveys, like those from Stack Overflow, highlight how developers perceive AI frameworks, examining their usage and preference trends within the community. For instance, a substantial proportion of developers reported a preference for frameworks that facilitate ease of use and effective support, emphasizing practical applications over theoretical robustness.

### Conclusion

In summary, the landscape of AI frameworks as of 2023 illustrates a continuing evolution toward flexibility, user-centric design, and integration of various technologies. Selecting the right framework encompasses more than just performance rankings; it involves understanding the specific needs of a project, evaluating metrics that matter, and considering broader trends in AI technology. TensorFlow, PyTorch, Keras, and Scikit-Learn represent stalwarts in the field, distinguished by their performance, community engagement, and ease of use. As AI continues to develop rapidly, these frameworks and their protocols will inevitably adapt, prompting ongoing discussions about their efficacy and future applications.

### Sources

[1] Top 6 AI Frameworks That Developers Should Learn in 2023 — https://www.linkedin.com/pulse/top-6-ai-frameworks-developers-should-learn-2023-souvik-bose  
[2] The Top 16 AI Frameworks and Libraries: A Beginner's Guide — https://www.datacamp.com/blog/top-ai-frameworks-and-libraries  
[3] Top 10 AI Libraries and Frameworks Every Beginner Should Know — https://medium.com/predict/top-10-ai-libraries-and-frameworks-every-beginner-should-know-b477933afd03  
[4] Comparing AI Frameworks: How to Decide If You Need ... — https://secureframe.com/blog/ai-frameworks  

</details>

# Executive Summary

This report provides an in-depth analysis of the leading AI agent frameworks in 2023, including AutoGen, LangChain, and CrewAI. Each framework is evaluated for its strengths, target user demographics, and overall suitability for various applications. A comparative analysis reveals that the diversity in capabilities among these frameworks highlights the necessity of aligning framework choices with specific user needs and operational goals, ultimately paving the way for enhanced AI integration across industries.

# Introduction

The rapid advancement of artificial intelligence has led to the emergence of numerous AI agent frameworks tailored for diverse applications. With these frameworks, developers can create more efficient and capable AI systems. This report explores the top frameworks currently dominating the market, focusing on their functionalities and the specific needs they fulfill.

# Overview of AI Agent Frameworks

AI agent frameworks serve as foundational tools that facilitate the development and deployment of AI functionalities, enabling developers to build applications that can understand, process, and respond to user inputs and operational commands effectively.

## Importance of AI Agent Frameworks in Modern Development

Today, selecting the right AI agent framework is crucial for ensuring the success of AI integration in various sectors. As technologies evolve, frameworks must support automation, user engagement, and a seamless development experience.

## Criteria for Evaluating AI Frameworks

When determining the most suitable AI agent frameworks, several criteria must be considered:
- **Usability**: How accessible is the framework for different user expertise levels?
- **Performance**: Frameworks should enable efficient computation and responsiveness.
- **Scalability**: Flexibility to adapt to growing user needs is vital.
- **Community Support**: Strong community backing fosters innovation and troubleshooting.

# Key AI Agent Frameworks

## AutoGen
- **Overview and Adoption**: Developed by Microsoft and launched in September 2023, AutoGen focuses on multi-agent collaborative tasks. Organizations like Novo Nordisk have already adopted it, indicating its market appeal.
- **Strengths and Applications**: AutoGen excels in handling complex interactions, making it suitable for enterprise applications that require robust collaboration among agents. This framework's event-driven architecture is particularly noteworthy for its effectiveness in multi-agent scenarios.

## LangChain
- **Features and Capabilities**: LangChain is recognized for its powerful tools for building AI agents utilizing large language models (LLMs). It allows developers to construct sophisticated workflows by chaining prompts and models seamlessly.
- **User Demographics**: Its strength in enabling intricate workflows positions LangChain as a strong choice for enterprise-focused applications, appealing to developers needing flexibility in design.

## CrewAI
- **Accessibility and No-Code Solutions**: CrewAI prioritizes user-friendly design, featuring a no-code interface that allows users to deploy AI agents without extensive programming knowledge. This makes it appealing for smaller businesses and startups.
- **Target Audience and Use Cases**: Equipped with ready-made templates, CrewAI is particularly effective for those looking to prototype quickly and engages users with varying levels of technical expertise.

# Comparative Analysis of AI Agent Frameworks

## Usability and Learning Curve
- **Beginner-Friendly Options vs. Advanced Capabilities**: While frameworks like AutoGen and CrewAI cater to those new to AI development with their approachable interfaces, LangChain is designed for users with advanced skills, enabling complex task management.

## Performance Metrics and Evaluation Criteria
- **Speed**: Essential for real-time applications, frameworks must deliver quick processing times.
- **Scalability**: Frameworks must accommodate larger datasets and adapt as operational complexity grows.
- **Community Support**: A robust community provides essential resources, enhancing user experience and problem resolution.

# Emerging Frameworks and Trends

## New Players in the AI Framework Space
- Recent analysis also highlighted the emergence of platforms such as MetaGPT, which provide promising alternatives for specific niches not fully covered by mainstream frameworks.

## Modular Architecture and Integration of Tools
- A combined and modular approach is becoming a trend, allowing developers to leverage the strengths of multiple frameworks.

# Conclusion

As AI frameworks continue to evolve, it is critical to stay informed about the latest developments to leverage the best tools tailored for specific needs. The current landscape showcases a vibrant selection of frameworks, each offering unique strengths that cater to various user needs.

# Recommendations

To ensure successful AI integration, organizations should closely assess their specific requirements and select frameworks that align with their operational goals. For broader projects requiring multiple functionalities, frameworks like LangChain and AutoGen are recommended. Conversely, for rapid prototyping and less complex need, CrewAI represents a compelling option.

# Follow-Up Research Topics
- The future of AI agent frameworks: Trends and predictions.
- Comparative analysis of modular versus monolithic architectures in AI.
- User adoption patterns of AI frameworks in small vs. large enterprises.
- The impact of community support on the effectiveness of AI frameworks.
- Evaluating the relationship between framework performance metrics and real-world applications.

### References

1. [Top 9 AI Agent Frameworks as of July 2025 | Shakudo](https://www.shakudo.io/blog/top-9-ai-agent-frameworks)
2. [Top 7 Free AI Agent Frameworks | Botpress](https://botpress.com/blog/ai-agent-frameworks)
3. [The Best AI Agents in 2025: Tools, Frameworks, and Platforms Compared | DataCamp](https://www.datacamp.com/blog/best-ai-agents)
4. [AI Agent Frameworks: Choosing the Right Foundation for Your Business | IBM](https://www.ibm.com/think/insights/top-ai-agent-frameworks)

In [30]:
# # ======================================================================================
# # 10) Email test
# # ======================================================================================


# from agents import Runner
# import html

# query = "List of AI Agent frameworks in 2025"
# subject = f"Deep Research – {query}"

# html_body = f"""
# <h1>{html.escape(subject)}</h1>
# <hr/>
# <pre style="white-space:pre-wrap;font-family:ui-monospace, SFMono-Regular, Menlo, Consolas;">
# {html.escape(report.markdown_report)}
# </pre>
# """

# await Runner.run(
#     email_agent,
#     f"""Send this as an HTML email using your tool.
# subject: {subject}
# html_body:
# <<<HTML
# {html_body}
# HTML"""
# )


In [29]:

# ======================================================================================
# 11) Gradio demo (optional)
# ======================================================================================


"""
Gradio UI for your Deep Research pipeline.

This app assumes you already have the orchestration helpers defined in the same Python
process (plan_searches, perform_searches, write_report, run_research) or imported from
your module. If you ran your earlier code in the same session (e.g., Jupyter), this
file can import them directly with a `from your_module import run_research`.

If you prefer a fully-standalone app, see the STANDALONE NOTE near the bottom for
how to swap in an import.
"""

import asyncio
import os
from datetime import datetime
from typing import List, Tuple

import gradio as gr

try:
    run_research  # type: ignore[name-defined]
except NameError:
    raise RuntimeError(
        "run_research(query) is not defined in this process.\n"
        "Import it here, e.g.: from deep_research import run_research\n"
        "Or run this app in the same kernel where you defined run_research."
    )


def _plan_to_rows(plan) -> List[list]:
    """Turn WebSearchPlan into rows for a DataFrame-friendly structure (list of lists)."""
    rows: List[list] = []
    try:
        for i, it in enumerate(plan.searches, 1):
            rows.append([i, getattr(it, "query", ""), getattr(it, "reason", "")])
    except Exception:
        pass
    return rows


def _citations_to_md(report) -> str:
    """Render final citations as Markdown numbered list."""
    if not getattr(report, "final_citations", None):
        return ""
    lines = ["\n### Sources\n"]
    for idx, c in enumerate(report.final_citations, 1):
        title = getattr(c, "title", None) or "Source"
        url = getattr(c, "url", "")
        if url:
            lines.append(f"{idx}. [{title}]({url})")
        else:
            lines.append(f"{idx}. {title}")
    return "\n".join(lines)


def _write_markdown_to_file(md: str, folder: str = ".") -> str:
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    path = os.path.join(folder, f"deep_research_{ts}.md")
    with open(path, "w", encoding="utf-8") as f:
        f.write(md)
    return path


async def stream_research(chat_history: List[Tuple[str, str]], topic: str, progress=gr.Progress(track_tqdm=False)):
    """
    Chat-like streaming handler. Yields intermediate updates as tuples compatible with Chatbot(type="tuples").
    """
    history = list(chat_history or [])
    history.append((topic, "Starting deep research..."))
    yield history, [], "", "", "", None

    progress(0.05, desc="Planning searches…")
    history[-1] = (topic, "Planning searches…")
    yield history, [], "", "", "", None

    progress(0.12, desc="Running pipeline… (this may take a bit)")
    plan, mini_summaries, report = await run_research(topic)  # type: ignore[name-defined]

    progress(0.55, desc="Formatting results…")
    plan_rows = _plan_to_rows(plan)
    history[-1] = (topic, "Searches planned. Gathering and synthesizing…")
    yield history, plan_rows, "", "", "", None

    if mini_summaries:
        parts = ["## Mini-summaries"]
        for i, s in enumerate(mini_summaries, 1):
            parts.append(f"<details><summary><b>Summary {i}</b></summary>\n\n{s}\n\n</details>")
        mini_md = "\n\n".join(parts)
    else:
        mini_md = "(No mini-summaries produced)"

    yield history, plan_rows, mini_md, "", "", None

    report_md = getattr(report, "markdown_report", "")
    sources_md = _citations_to_md(report)

    full_md = report_md + ("\n\n" + sources_md if sources_md else "")
    file_path = _write_markdown_to_file(full_md)

    history[-1] = (topic, "Done ✅")
    progress(1.0, desc="Complete")
    yield history, plan_rows, mini_md, report_md, sources_md, file_path


with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🔎 Deep Research (Gradio)
        Enter a topic and I'll plan searches, gather snippets, and synthesize a long-form report.
        """
    )

    with gr.Row():
        chatbot = gr.Chatbot(label="Research Chat", height=350, type="tuples")
    
    with gr.Row():
        topic = gr.Textbox(placeholder="e.g., Latest AI Agent frameworks in 2025", label="Topic", lines=1)
        go = gr.Button("Run Research", variant="primary")

    with gr.Row():
        plan_table = gr.Dataframe(headers=["#", "query", "reason"], label="Planned web searches", interactive=False)

    mini_md = gr.Markdown(label="Mini-summaries")
    report_md = gr.Markdown(label="Synthesis report (Markdown)")
    sources_md = gr.Markdown(label="Sources")

    with gr.Row():
        dl = gr.DownloadButton(label="Download Markdown", value=None)

    state = gr.State([])

    go.click(
        fn=stream_research,
        inputs=[state, topic],
        outputs=[chatbot, plan_table, mini_md, report_md, sources_md, dl],
        show_progress=True,
    ).then(lambda h: h, inputs=chatbot, outputs=state)

    gr.Examples([
        ["Latest AI Agent frameworks in 2025"],
        ["State of multimodal RAG for e-commerce"],
        ["Carbon accounting tools for utilities in 2025"]
    ], inputs=[topic])

if __name__ == "__main__":
    demo.launch(share = True)

"""
STANDALONE NOTE:
Replace the `try/except` block with:

    from your_module import run_research

Ensure that `your_module.run_research` matches the signature used above.
"""


C:\Users\saket\AppData\Local\Temp\ipykernel_10844\157723431.py:119: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Research Chat", height=350, type="tuples")


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://c9ea5cb89185c086cc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


'\nSTANDALONE NOTE:\nReplace the `try/except` block with:\n\n    from your_module import run_research\n\nEnsure that `your_module.run_research` matches the signature used above.\n'

Planning...
ℹ️  Planned search phrases:
  1. OpenAI agents SDK documentation — To find the official documentation and resources for using OpenAI's agents SDK.
  2. OpenAI agents SDK features updates — To gather information on the latest features and updates related to OpenAI agents SDK.
  3. OpenAI agents SDK tutorials community — To explore community discussions, tutorials, and examples of how to implement the OpenAI agents SDK in projects.
Searching...
🔎 Searching: OpenAI agents SDK documentation
🔎 Searching: OpenAI agents SDK features updates
🔎 Searching: OpenAI agents SDK tutorials community
✅ Done: OpenAI agents SDK documentation
✅ Done: OpenAI agents SDK features updates
✅ Done: OpenAI agents SDK tutorials community
Synthesizing...
